# Analysis of InferSent Model

In [1]:
import torch
from train import NLINet
from data import SNLIData
from utils import download_file_from_google_drive

## Download pretrained model

Use the following file IDs to download the corresponding models.


1. MeanEmbedding: `1q4ZRin0tKohQ504fi5HVkjDiolLZjuxg`
2. LSTM: `1lwClDt1cNaOtOo5h-bTx-rWr7ePeIyIO`
3. BiLSTM: `1zPZzm1EECkLdcbQ_SShhYPOrBXNu_zvz`
4. BiLSTM-maxpool: `12BzrDODCYjMZLhld1SFcyckwAa4Vj4fL`

Let's download the model checkpoint with file ID from above.

In [ ]:
download_file_from_google_drive('12BzrDODCYjMZLhld1SFcyckwAa4Vj4fL','bilstm-maxpool.ckpt')

## Evalute on SNLI test data

In [3]:
from eval import snli, senteval, process_senteval_result

checkpoint_path = 'bilstm-maxpool.ckpt'

snli(checkpoint_path)

params_senteval = {
    'task_path': './SentEval/data/',
    'usepytorch': True,
    'kfold': 5,
    'classifier': {
        'nhid': 0,
        'optim': 'rmsprop',
        'batch_size': 128,
        'tenacity': 3,
        'epoch_size': 2
    }
}

# result_dict = senteval(checkpoint_path,
#           params_senteval=params_senteval)
# process_senteval_result(result_dict)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Testing: 0it [00:00, ?it/s]

Final Test Accuracy: 0.8598330618892508
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc_epoch': 0.8598330618892508,
 'test_acc_step': 0.8598330616950989,
 'test_loss': 0.37223106622695923}
--------------------------------------------------------------------------------
[{'test_acc_step': 0.8598330616950989, 'test_loss': 0.37223106622695923, 'test_acc_epoch': 0.8598330618892508}]


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NLINet.load_from_checkpoint('bilstm-maxpool.ckpt').to(device)
model.eval()

NLINet(
  (model): InferSent(
    (encoder): BiLSTMEncoder(
      (embedding): Embedding(33893, 300)
      (linear): Linear(in_features=300, out_features=1028, bias=True)
      (relu): ReLU()
      (projection): Sequential(
        (0): Embedding(33893, 300)
        (1): Linear(in_features=300, out_features=1028, bias=True)
        (2): ReLU()
      )
      (lstm): LSTM(1028, 1028, batch_first=True, bidirectional=True)
    )
    (classifier): Classifier(
      (lin1): Linear(in_features=8224, out_features=512, bias=True)
      (lin2): Linear(in_features=512, out_features=512, bias=True)
      (lin3): Linear(in_features=512, out_features=3, bias=True)
      (relu): ReLU()
      (net): Sequential(
        (0): Linear(in_features=8224, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): ReLU()
        (4): Linear(in_features=512, out_features=3, bias=True)
      )
    )
  )
  (criterion): CrossEntropyLoss()
)

In [5]:
data = SNLIData(batch_size=128)
vocab = data.get_vocab()

## Making predictions

In [36]:
import spacy

def tokenize(text):
    spacy_en = spacy.load('en_core_web_sm')
    return [tok.text for tok in spacy_en.tokenizer(text)]

def get_encoded_text(text, max_len=12, verbose=True):
    
    # Get vocab index of each tokenized words.
    tokens = tokenize(text)
    sent_idxs = [vocab[tok] for tok in tokens]
    
    # Apply padding for consistent length.
    for _ in range(len(sent_idxs) + 1, max_len + 1):
        sent_idxs.append(vocab["<pad>"])
    
    # Print tokens and idxs for debugging.
    if verbose:
        print(sent_idxs)
        print([data.text.vocab.itos[i] for i in sent_idxs])
    
    embed = torch.LongTensor([sent_idxs]).to(device)
    return model.model.encode(embed).long()

def get_label(idx):
    label_map = {0:'Entailment',1:'Contradiction',2:'Neural'}
    return label_map[idx]

Now let's provide our own premise and hypothesis and see what the model predicts.

In [45]:
# entailment
premise = 'A soccer game with multiple males playing.'
hypothesis = 'Some men are playing a sport.'

In [47]:
# contradiction
premise = 'A black race car starts up in front of a crowd of people.'
hypothesis = 'A man is driving down a lonely road.'

In [43]:
# neutral
premise = 'An older and younger man smiling.'
hypothesis = 'Two men are smiling and laughing at the cats playing on the floor.'

In [48]:
verbose = True
premise_enc = get_encoded_text(premise, verbose=verbose)
hypothesis_enc = get_encoded_text(hypothesis, verbose=verbose)

out = model.model(((premise_enc,hypothesis_enc),_))
print('Model output:', out)
pred = get_label(torch.argmax(torch.nn.functional.log_softmax(out, 1), 1).item())

print('Prediction:', pred)

[0, 30, 181, 106, 3081, 55, 5, 45, 11, 2, 86, 11, 15, 3]
['<unk>', 'black', 'race', 'car', 'starts', 'up', 'in', 'front', 'of', 'a', 'crowd', 'of', 'people', '.']
[0, 7, 6, 324, 39, 2, 3524, 154, 3, 1, 1, 1]
['<unk>', 'man', 'is', 'driving', 'down', 'a', 'lonely', 'road', '.', '<pad>', '<pad>', '<pad>']
Model output: tensor([[ 3.4195, -3.2012, -1.3253]], grad_fn=<AddmmBackward>)
Prediction: Entailment
